In [26]:
%run "sql.ipynb"
import math
from scipy.stats import sem
from scipy import stats
x_sist = 0.5
alpha = 0.95
avg_x = {}
sigma = {}
SEM = {}
x_sl = {}
abs_x = {}
otn_x = {}
for i in dict_lab:
    n = len(dict_lab[i])
    avg_x[i] = np.mean(dict_lab[i])
    sigma[i] = np.std(dict_lab[i], ddof=1)
    SEM[i] = np.std(dict_lab[i], ddof=1) / np.sqrt(np.size(dict_lab[i]))
    x_sl[i] = stats.t.ppf((1 + alpha)/2, n-1) * SEM[i]
    abs_x[i] = math.sqrt(x_sist ** 2 + x_sl[i] ** 2)
    otn_x[i] = abs_x[i]/avg_x[i]
print(f"В этом блоке найдены:")
print(f"Количество измерений n: {n}")
print("Средние значения измеренных величин <Xi>:", avg_x, sep='\n')
print("Средние квадратичные погрешности отдельных результатов измерений Sn:", sigma, sep='\n')
print("Средние квадратичные погрешности среднего арифметического S:", SEM, sep='\n')
print("Случайные погрешности ΔXсл:", x_sl, sep='\n')
print("Абсолютные погрешности ΔX:", abs_x, sep='\n')
print("Относительные погрешности δX:", otn_x, sep='\n')

Connection to PostgreSQL DB successful
В этом блоке найдены:
Количество измерений n: 12
Средние значения измеренных величин <Xi>:
{'v': 12.508333333333333, 'u': 5.075, 's': 2.2583333333333333, 'h': 10.65}
Средние квадратичные погрешности отдельных результатов измерений Sn:
{'v': 0.07929614610987562, 'u': 0.07537783614444091, 's': 0.07929614610987579, 'h': 0.06741998624632396}
Средние квадратичные погрешности среднего арифметического S:
{'v': 0.022890825651118295, 'u': 0.02175970699446223, 's': 0.022890825651118343, 'h': 0.019462473604038005}
Случайные погрешности ΔXсл:
{'v': 0.05038236756015747, 'u': 0.047892792182564516, 's': 0.05038236756015758, 'h': 0.042836615580993755}
Абсолютные погрешности ΔX:
{'v': 0.5025319720783612, 'u': 0.5022884823913866, 's': 0.5025319720783612, 'h': 0.50183162079968}
Относительные погрешности δX:
{'v': 0.040175773916990905, 'u': 0.09897309997859834, 's': 0.22252338247012304, 'h': 0.04712033998119061}


In [27]:
dx = []
import sympy as sym
for i in dict_lab:
    globals()[i] = sym.Symbol(i)
    dx.append(sym.Symbol(i))
print(f"Запишите функцию {find_val} в переменную f в блоке ниже. Она должна содержать параметры {dx}, по которым будет производиться дифференцирование")

Запишите функцию J в переменную f в блоке ниже. Она должна содержать параметры [v, u, s, h], по которым будет производиться дифференцирование


In [28]:
f = h*s + v**2/u**0.5
print(f"Вы записали функцию f = {f}")
print(f"Вставьте значение переменной f в переменную avg_f в следующем блоке")

Вы записали функцию f = h*s + v**2/u**0.5
Вставьте значение переменной f в переменную avg_f в следующем блоке


In [29]:
print(f"Найдем среднее значение {find_val}, используя средние значения измеренных величин:")
for i in dict_lab:
    print(f"{i}:{avg_x[i]}")
    globals()[i] = avg_x[i]

avg_f = h*s + v**2/u**0.5

print(f"Среднее значение {find_val} = {avg_f}")

Найдем среднее значение J, используя средние значения измеренных величин:
v:12.508333333333333
u:5.075
s:2.2583333333333333
h:10.65
Среднее значение J = 93.5026283210546


In [30]:
print(f"Найдем значения производных df/dxi для каждого измеренного значения xi:")
derivative_f = []
dfdx = []
for i in range(len(val_names)):
    df = f.diff(dx[i])
    derivative_f.append(df)
    f_i = sym.lambdify([j for j in dx], df)
    dfdx.append(f_i(v, u, s, h))
print(f"Найденные значения производных df/dxi для каждого измеренного значения xi: {dfdx}")

Найдем значения производных df/dxi для каждого измеренного значения xi:
Найденные значения производных df/dxi для каждого измеренного значения xi: [11.104817319822189, -6.842500327197498, 10.65, 2.2583333333333333]


In [31]:
print(f"Подставим df/dxi и абсолютные погрешности xi в формулу для нахождения абсолютной погрешности косвенного измерения")
abs_f = 0
for i, v in enumerate(dict_lab):
    abs_f += (dfdx[i] * abs_x[v]) ** 2
abs_f = math.sqrt(abs_f)
otn_f = abs_f/avg_f
print(f"Абсолютная погрешность Δ{find_val} = {abs_f}")
print(f"Относительная погрешность δ{find_val} = {otn_f} или {round(otn_f * 100, 2)}%")

Подставим df/dxi и абсолютные погрешности xi в формулу для нахождения абсолютной погрешности косвенного измерения
Абсолютная погрешность ΔJ = 8.537126121754222
Относительная погрешность δJ = 0.09130359515072434 или 9.13%


In [32]:
print(f"Добавляем найденные значения в БД {result_table}")
connection = create_connection("postgres", "postgres", "1234", "localhost", "5432")
for i in dict_lab:
    update = f"""
UPDATE {result_table}
SET
avg_x = '{avg_x[i]}',
abs_x = '{abs_x[i]}',
otn_x = '{otn_x[i]}',
s = '{SEM[i]}'
WHERE
value = '{i}'
"""
    update_table = execute_query(connection, update)

update = f"""
UPDATE {result_table}
SET
avg_x = '{avg_f}',
abs_x = '{abs_f}',
otn_x = '{otn_f}'
WHERE
value = '{find_val}'
"""
update_table = execute_query(connection, update)

Добавляем найденные значения в БД results_lab1
Connection to PostgreSQL DB successful
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
